In [51]:
import os, json
import pandas as pd
import numpy as np
import glob
pd.set_option('display.max_columns', None)
dir = 'data'
path = os.path.join(dir, '*.json')
file_list = glob.glob(path)
cat_list = [l.split('.')[0] for l in os.listdir(dir)]

dfs = list()

for file, cat in dict(zip(file_list, cat_list)).items():
    with open(file, 'r') as f:
        data = json.load(f)
    channel_id, stats = data.popitem()
    pchannel_stats = stats["channel_statistics"]
    video_stats = stats["video_data"]
    sorted_vids = sorted(video_stats.items(), key=lambda item: int(item[1]["viewCount"]),reverse=True)
    stats = []
    for vid in sorted_vids:
        video_id = vid[0]
        title = vid[1]["title"]
        views = vid[1]["viewCount"]
        likes = vid[1]["likeCount"]
        duration = vid[1]['duration']
        try:
            tags = vid[1]['tags']
        except:
            pass
        description = vid[1]['description']
        #dislikes = vid[1]["dislikeCount"]
        try:
            comments = vid[1]["commentCount"]
        except:
            pass
        channel = vid[1]['channelTitle']
        stats.append([title,views, likes, comments, duration, tags, description, channel, cat])
    vid_df = pd.DataFrame(stats, columns=["title","views", "likes","comments", 'duration','tag','description', 'channel', 'category'])
    dfs.append(vid_df)
df = pd.concat(dfs, ignore_index=True)

In [52]:
df

,title,views,likes,comments,duration,tag,description,channel,category
0,Cobra vs. Mongoose | National Geographic,92381617,142210,21868,PT1M57S,"[Vs., national geographic, nat geo, natgeo, an...",How does a mongoose stand up to a cobra? You m...,National Geographic,national_geographic
1,See a Sea Turtle Devour a Jellyfish Like Spagh...,45929511,213498,17407,PT1M17S,"[national geographic, nat geo, natgeo, animals...",A marine biologist captured footage of a green...,National Geographic,national_geographic
2,109-Year-Old Veteran and His Secrets to Life W...,39705519,1065232,72082,PT12M39S,"[Richard Overton, Veteran, World War II, super...","Meet Richard Overton, America's oldest veteran...",National Geographic,national_geographic
3,Two Blind Sisters See for the First Time | Sho...,31249322,613243,31151,PT4M32S,"[Short film showcase, national geographic, nat...","Sonia and Anita, two sisters living in India, ...",National Geographic,national_geographic
4,A Fighting Tiger | Shark Men,27628894,96629,2757,PT2M54S,"[National Geographic, Shark Men: Trouble In Pa...",The Shark Men land a very aggressive tiger sha...,National Geographic,national_geographic
...,...,...,...,...,...,...,...,...,...
1488,DON'T Cross This Hidden Laser!,70790,1832,134,PT8M7S,"[thekingofrandom, tkor, random happens, grant ...",Today we're upgrading the security here at TKO...,TKOR,tkor
1489,This Magic Sand Repels Water #shorts,59316,2162,43,PT58S,"[tkor, shorts, proto putty, shoes]",Check out the full video here: https://youtu.b...,TKOR,tkor
1490,This Visual Phenomenon Looks UNREAL!,50397,1580,29,PT23S,"[thekingofrandom, tkor, random happens, grant ...",Subscribe to TKOR: https://www.youtube.com/c/T...,TKOR,tkor
1491,EMERGENCY Burrito Bowl,36102,1275,197,PT12M53S,"[thekingofrandom, tkor, random happens, grant ...",Thanks to The United States Marine Corp for Sp...,TKOR,tkor
